In [ ]:
pip install openai

In [7]:
import os
import requests
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
pubmed_api_key = os.getenv("PUBMED_API_KEY")

def chat_with_llm(user_input):
    client = AzureOpenAI(
        azure_endpoint = azure_endpoint, 
        api_key = azure_api_key,  
        api_version = "2024-02-01"
    )

    response = client.chat.completions.create(
        model="gpt-4o-doc-search", # model = "deployment_name".
        messages=[
            {"role": "system", "content": '당신은 의사들이 원하는 의학 논문을 쉽게 찾아주는 도우미입니다. 당신은 의사가 원하는 키워드로 논문을 찾을 수 있게 하기 위해서 의사가 논문을 찾고, 원하는 키워드를 말한다면 말 중에 논문으로 검색할 수 있는 키워드만 추출하고 대괄호 안에 넣은 영어 키워드만 답변합니다. 만약 논문에 대한 대화가 아니라면 그냥 의사의 대화에 맞게 말해주세요.'},
            {"role": "user", "content": user_input},
        ]
    )
    assistant_msg = response.choices[0].message.content

    # APi를 호출하기 위하여 대괄호가 포함되었는지 확인
    if "[" in assistant_msg and "]" in assistant_msg:
        
        # API 호출
        api_response = call_pubmed_api(assistant_msg)

        # api_response를 다시 LLM에 보내기
        assistant_reply = client.chat.completions.create( 
            model="gpt-4o-doc-search",
            messages=[
                {"role": "system", "content": '당신은 (1)과 같은 형식의 논문의 정보를 받으면 (2)와 같이 제시를 합니다. --- (1) Title: Disparities in cancer-related financial toxicity across economically diverse provinces in China: A multi-center cross-sectional study Authors: Xu B, So WK, Choi KC, Huang Y, Liu M, Qiu L, Tan J, Tao H, Yan K, Yang F Publication Date: 2024 Dec 9 Journal: Asia-Pacific Journal of Oncology Nursing DOI: https://doi.org/10.1016/j.apjon.2024.100636 (2) Asia-Pacific Journal of Oncology Nursing 저널에 "중국의 경제적으로 다양한 지방에서 발생하는 암 관련 재정적 부담의 불평등"라는 제목의 논문이 2024년 12월 9일에 실렸어요. 저자는 Xu B와 So WK, Choi KC, Huang Y 등입니다. 자세한 내용은 https://doi.org/10.1016/j.apjon.2024.100636 에서 확인하세요.'},
                {"role": "user", "content": api_response},
            ]
        )
        # 정제된 논문 정보를 사용자에게 보내기
        print(assistant_reply.choices[0].message.content)
        return assistant_reply.choices[0].message.content
    
    else:
        # 일반 내용을 사용자에게 보내기
        return assistant_msg

def call_pubmed_api(keyword):
    
    db = 'pmc'  # 'pubmed' or 'pmc'

    # ESEARCH API URL
    esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

    # ESEARCH 요청 파라미터 설정
    esearch_params = {
        'db': db,                     # 데이터베이스에서 검색
        'term': keyword,              # 검색어
        'retmode': 'json',            # 반환 형식 (JSON)
        'api_key': pubmed_api_key,    # API 키
        'retmax': 5,                  # 검색 결과 최대 10개
        'sort' : 'relevance'          # 관련성 높은 순으로 검색
    }

    # ESEARCH 요청 보내기
    esearch_response = requests.get(esearch_url, params=esearch_params)

    # ESEARCH 응답 결과 처리
    if esearch_response.status_code == 200:
        esearch_data = esearch_response.json()
        pmids = esearch_data.get('esearchresult', {}).get('idlist', [])
        
        if not pmids:
            print("검색된 PMID가 없습니다.")
        else:
            # ESUMMARY API URL
            esummary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"

            # ESUMMARY 요청 파라미터 설정
            esummary_params = {
                'db': db,                       # 데이터베이스
                'id': ','.join(pmids),          # 검색된 PMID들
                'retmode': 'json',              # 반환 형식 JSON
                'api_key': pubmed_api_key       # API 키
            }

            # ESUMMARY 요청 보내기
            esummary_response = requests.get(esummary_url, params=esummary_params)

            all_paper_info = []

            # ESUMMARY 응답 결과 처리
            if esummary_response.status_code == 200:
                esummary_data = esummary_response.json()
                
                for pmid in pmids:
                    paper = esummary_data['result'][pmid]

                    # 주요 정보 추출
                    title = paper['title']
                    authors = [author['name'] for author in paper['authors']]
                    pubdate = paper['pubdate']
                    journal = paper['fulljournalname']
                    doi = next((id['value'] for id in paper['articleids'] if id['idtype'] == 'doi'), "DOI 정보 없음")

                    # 정보 출력
                    paper_info = f"""
                    Title: {title}
                    Authors: {', '.join(authors)}
                    Publication Date: {pubdate}
                    Journal: {journal}
                    DOI: https://doi.org/{doi}
                    """

                    all_paper_info.append(paper_info.strip())

            else:
                print("ESUMMARY 요청 오류:", esummary_response.status_code)

            # 논문 정보를 하나의 문자열로 합치기
            final_paper_info = "\n\n".join(all_paper_info)

            return final_paper_info

    else:
        print("ESEARCH 요청 오류:", esearch_response.status_code)

# 밑에서 챗봇에 입력할 수 있습니다!

In [8]:
chat_with_llm("당뇨 경구약제 결정에 대한 가이드라인")

The Journal of Clinical Endocrinology and Metabolism 저널에 "성인에서의 중심 성선뇌하수체 기능저하증의 진단 및 관리"라는 제목의 논문이 2022년 6월 30일에 실렸어요. 저자는 Tomkins M와 Lawless S, Martin-Grace J 등입니다. 자세한 내용은 https://doi.org/10.1210/clinem/dgac381 에서 확인하세요.

Cureus 저널에 "갈증이 없어졌을 때: 파열된 전교통동맥 동맥류의 코일 색전술 후 발생한 무갈증 소변과다증의 증례 보고 및 문헌 검토"라는 제목의 논문이 2024년 7월 10일에 실렸어요. 저자는 Barnett MJ와 Patel G, Lekprasert P 등입니다. 자세한 내용은 https://doi.org/10.7759/cureus.64207 에서 확인하세요.

The Journal of Clinical Endocrinology and Metabolism 저널에 "환자 접근법: 코펩틴 검사 효용성"라는 제목의 논문이 2022년 2월 8일에 실렸어요. 저자는 Christ-Crain M와 Refardt J, Winzeler B 등입니다. 자세한 내용은 https://doi.org/10.1210/clinem/dgac070 에서 확인하세요.

Endocrine practice : official journal of the American College of Endocrinology and the American Association of Clinical Endocrinologists 저널에 "미국 임상 내분비학회 임상 실무 지침: 당뇨병 포괄적 관리 계획 개발—2022 업데이트"라는 제목의 논문이 2022년 8월 11일에 실렸어요. 저자는 Blonde L와 Umpierrez GE, Reddy SS 등입니다. 자세한 내용은 https://doi.org/10.1016/j.eprac.2022.08.002 에서 확인하세요.

Clinical Endocrinology 저널에

'The Journal of Clinical Endocrinology and Metabolism 저널에 "성인에서의 중심 성선뇌하수체 기능저하증의 진단 및 관리"라는 제목의 논문이 2022년 6월 30일에 실렸어요. 저자는 Tomkins M와 Lawless S, Martin-Grace J 등입니다. 자세한 내용은 https://doi.org/10.1210/clinem/dgac381 에서 확인하세요.\n\nCureus 저널에 "갈증이 없어졌을 때: 파열된 전교통동맥 동맥류의 코일 색전술 후 발생한 무갈증 소변과다증의 증례 보고 및 문헌 검토"라는 제목의 논문이 2024년 7월 10일에 실렸어요. 저자는 Barnett MJ와 Patel G, Lekprasert P 등입니다. 자세한 내용은 https://doi.org/10.7759/cureus.64207 에서 확인하세요.\n\nThe Journal of Clinical Endocrinology and Metabolism 저널에 "환자 접근법: 코펩틴 검사 효용성"라는 제목의 논문이 2022년 2월 8일에 실렸어요. 저자는 Christ-Crain M와 Refardt J, Winzeler B 등입니다. 자세한 내용은 https://doi.org/10.1210/clinem/dgac070 에서 확인하세요.\n\nEndocrine practice : official journal of the American College of Endocrinology and the American Association of Clinical Endocrinologists 저널에 "미국 임상 내분비학회 임상 실무 지침: 당뇨병 포괄적 관리 계획 개발—2022 업데이트"라는 제목의 논문이 2022년 8월 11일에 실렸어요. 저자는 Blonde L와 Umpierrez GE, Reddy SS 등입니다. 자세한 내용은 https://doi.org/10.1016/j.eprac.2022.08.002 에서 확인하세요.\n\nClinical Endocrin